# <span style="color:DarkSeaGreen">Lab 3 - AgentCore</span>
- this notebook creates the following:
  - AgentCore environment to deploy the Strands Agents
    - Uses the Amazon Bedrock AgentCore starter toolkit

  - it shows you how to use the Amazon Bedrock AgentCore starter toolkit to deploy an agent to an AgentCore Runtime.
  - the starter toolkit is a Command Line Interface (CLI) toolkit that you can use to deploy AI agents to an AgentCore Runtime. You can use the toolkit with popular Python agent frameworks, such as LangGraph or Strands Agents. 
  - this workshops conitines with the Strands Agents created in lab 2

- Prerequisites:
  - https://aws.github.io/bedrock-agentcore-starter-toolkit/user-guide/runtime/quickstart.html
  - Model Access: Anthropic Claude Sonnet 4.0

In [ ]:
### START ###
### MAKE SURE YOU ARE IN THE RIGHT FOLDER WHERE THE REQUIREMENTS FILE IS LOCATED ###
### COPY FROM HERE ONLY IF RUNNING AS ONE COPY AND PASTE ###
# activate the virtual environment
source venv-agentcore/bin/activate
### COPY TO HERE ONLY IF RUNNING AS ONE COPY AND PASTE ###

### STOP ###
### MAKE SURE ABOVE VENV GETS ACTIVATED BEFORE RUNNING THE REST ###
# install the required packages - may need to specify the path here if not in the correct folder in terminal window
pip install -r requirements_lab3.txt
# pip install -r Documents/github/labs-sagemaker/jumpstart/requirements_lab3.txt
# verify the installation
pip list

### RESTART VSCODE TO PICKUP THE NEW VENV ###

In [ ]:
### STOP ###
### This command is for activating an environment that already exists, its for use in a terminal window if you need it ###
source venv-agentcore/bin/activate
pip list

# use pip freeze if you prefer for friendly format
### ALSO MAKE SURE YOU SELECT IT AS YOUR KERNEL FOR THIS JUPYTER NOTEBOOK ###

# <span style="color:DarkSeaGreen">AgentCore MCP Server</span>
- If you are using an environment that supports use of MCP Servers, you can install the AgentCore MCP Server to provide natural language interfaces for AgentCore development
- For example, this notebook was developed using the help of the AgentCore MCP Server by installing it into VSCode Amazon Q Chatbot
- https://aws.amazon.com/blogs/machine-learning/accelerate-development-with-the-amazon-bedrock-agentcore-mcpserver/

In [ ]:
Name: bedrock-agentcore-mcp-server
Transport: stdio
Command: uvx
Arguments: awslabs.amazon-bedrock-agentcore-mcp-server@latest

# Lab 3 Starts Here!

# <span style="color:DarkSeaGreen">AgentCore</span>

- We will deploy the 4 Strands agents and the Strands orchestration agent into AgentCore
  - We will use the Bedrock AgentCore Runtime starter toolkit
- AgentCore hosts them as containerized applications
  - Strands agents can be deployed to AgentCore as-is without converting them to Lambda functions
  - AgentCore uses a different deployment model:
    - Container-based deployment : Your Python scripts are packaged into containers and run on AgentCore Runtime
    - No Lambda conversion needed : The agents run as containerized applications, not serverless functions
    - Built-in orchestration : You can deploy multiple agents and route between them
- Make sure you have the relevant IAM role and permnissions for the agent
    - https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-permissions.html


# <span style="color:DarkSeaGreen">Setup</span>

In [ ]:
# region - we use us-east-1 as Bedrock is limited in other reasons
myRegion='us-east-1'

# iam
myRoleAgentCore="doit-agentcore-support-agentcore-role"
myPolicyAgentCore1="doit-agentcore-kb-support-agentcore-policy"
myRoleAgentCoreARN='RETRIEVED FROM ROLE BELOW ONCE CREATED'

print ('✅ Done! Move to the next cell ->')

In [ ]:
import boto3
import json
from certifi import where

# Configure boto3 to use certifi's certificates
sts_client = boto3.client('sts', verify=where())
myAccountNumber = sts_client.get_caller_identity()["Account"]
print(myAccountNumber)
print(sts_client.get_caller_identity()["Arn"])

print ('✅ Done! Move to the next cell ->')

In [ ]:
# iam
iam = boto3.client('iam', region_name=myRegion, verify=where())

print ('✅ Done! Move to the next cell ->')

In [ ]:
# define tags added to all services we create
myTags = [
    {"Key": "env", "Value": "non_prod"},
    {"Key": "owner", "Value": "doit_agentcore_lab"},
    {"Key": "project", "Value": "doit_agentcore_crypto"},
    {"Key": "author", "Value": "simon"},
]
myTagsDct = {
    "env": "non_prod",
    "owner": "doit_agentcore_lab",
    "project": "doit_agentcore_crypto",
    "author": "simon",
}

print ('✅ Done! Move to the next cell ->')

# <span style="color:DarkSeaGreen">IAM</span>

- agentcore iam
  - https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-permissions.html
- We only create the role required by AgentCore here
  - To run agent or tool in AgentCore Runtime you need an AWS Identity and Access Management execution role
  - The AgentCore Runtime execution role is an IAM role that AgentCore Runtime assumes to run an agent
  - https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-permissions.html#runtime-permissions-execution
- Check out the link if you need permissions for yourself to deploy to AgentCore
  - https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-permissions.html#runtime-permissions-starter-toolkit

In [ ]:
# define AgentCore Runtime execution json
policyJson = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "ECRImageAccess",
            "Effect": "Allow",
            "Action": ["ecr:BatchGetImage", "ecr:GetDownloadUrlForLayer"],
            "Resource": [f"arn:aws:ecr:{myRegion}:{myAccountNumber}:repository/*"],
        },
        {
            "Effect": "Allow",
            "Action": ["logs:DescribeLogStreams", "logs:CreateLogGroup"],
            "Resource": [
                f"arn:aws:logs:{myRegion}:{myAccountNumber}:log-group:/aws/bedrock-agentcore/runtimes/*"
            ],
        },
        {
            "Effect": "Allow",
            "Action": ["logs:DescribeLogGroups"],
            "Resource": [f"arn:aws:logs:{myRegion}:{myAccountNumber}:log-group:*"],
        },
        {
            "Effect": "Allow",
            "Action": ["logs:CreateLogStream", "logs:PutLogEvents"],
            "Resource": [
                f"arn:aws:logs:{myRegion}:{myAccountNumber}:log-group:/aws/bedrock-agentcore/runtimes/*:log-stream:*"
            ],
        },
        {
            "Sid": "ECRTokenAccess",
            "Effect": "Allow",
            "Action": ["ecr:GetAuthorizationToken"],
            "Resource": "*",
        },
        {
            "Effect": "Allow",
            "Action": [
                "xray:PutTraceSegments",
                "xray:PutTelemetryRecords",
                "xray:GetSamplingRules",
                "xray:GetSamplingTargets",
            ],
            "Resource": ["*"],
        },
        {
            "Effect": "Allow",
            "Resource": "*",
            "Action": "cloudwatch:PutMetricData",
            "Condition": {
                "StringEquals": {"cloudwatch:namespace": "bedrock-agentcore"}
            },
        },
        {
            "Sid": "GetAgentAccessToken",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetWorkloadAccessToken",
                "bedrock-agentcore:GetWorkloadAccessTokenForJWT",
                "bedrock-agentcore:GetWorkloadAccessTokenForUserId",
            ],
            "Resource": [
                f"arn:aws:bedrock-agentcore:{myRegion}:{myAccountNumber}:workload-identity-directory/default",
                f"arn:aws:bedrock-agentcore:{myRegion}:{myAccountNumber}:workload-identity-directory/default/workload-identity/agentName-*",
            ],
        },
        {
            "Sid": "BedrockModelInvocation",
            "Effect": "Allow",
            "Action": ["bedrock:InvokeModel", "bedrock:InvokeModelWithResponseStream"],
            "Resource": [
                "arn:aws:bedrock:*::foundation-model/*",
                f"arn:aws:bedrock:{myRegion}:{myAccountNumber}:*",
            ],
        },
    ],
}
# create policy
policy1 = iam.create_policy(
    PolicyName=myPolicyAgentCore1,
    PolicyDocument=json.dumps(policyJson),
    Description="Policy allowing AgentCore Runtime to run agents",
    Tags=[
        *myTags,
    ],
)

# trust policy for the role
roleTrust = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AssumeRolePolicy",
            "Effect": "Allow",
            "Principal": {"Service": "bedrock-agentcore.amazonaws.com"},
            "Action": "sts:AssumeRole",
            "Condition": {
                "StringEquals": {"aws:SourceAccount": f"{myAccountNumber}"},
                "ArnLike": {
                    f"aws:SourceArn": f"arn:aws:bedrock-agentcore:{myRegion}:{myAccountNumber}:*"
                },
            },
        }
    ],
}

# create role
role = iam.create_role(
    RoleName=myRoleAgentCore,
    AssumeRolePolicyDocument=json.dumps(roleTrust),
    Description="Service role for AgentCore Runtime",
    Tags=[
        *myTags,
    ],
)

# attach policies to role
iam.attach_role_policy(
    RoleName=role["Role"]["RoleName"], PolicyArn=policy1["Policy"]["Arn"]
)
myRoleAgentCoreARN = role["Role"]["Arn"]

print(f"🧠 Created Role ARN, replace <♻️ myRoleAgentCoreARN> below with your role ARN: {myRoleAgentCoreARN}")
print("✅ Done! Move to the next cell ->")

# <span style="color:DarkSeaGreen">Deployment of AgentCore</span>

- Execute the following in a terminal window

In [ ]:
# Either in a IDE terminal window, or your system terminal window, make sure the venv is activated
# If in a system terminal window, you will need to navigate to the folder where the venv is located
# eg typically labs-bedrock/agentcore
# then run the command below to activate the venv
source venv-agentcore/bin/activate

In [ ]:
# Then cd to the agents folder
cd agents

- The following will:
  - Generate a Dockerfile and .dockerignore
  - Create a .bedrock_agentcore.yaml configuration file
- ⚠️ If the CLI auth you are using is not in the environment where you want to deploy, make sure you specify the region as in below

In [ ]:
### Step 1: Configure Your Customer Support Agent
### https://aws.github.io/bedrock-agentcore-starter-toolkit/api-reference/cli.
### https://strandsagents.com/latest/documentation/docs/user-guide/deploy/deploy_to_bedrock_agentcore/
### Run this then copy the cell output to your terminal window to execute
print(f"agentcore configure --entrypoint Strands_Orchestration_Crypto_Agent.py -r {myRegion} -er {myRoleAgentCoreARN} --idle-timeout 300 --max-lifetime 900")

 The following will:
  - Deploy your agent to AgentCore Runtime
    - Build a Docker image with your agent code
    - Push the image to Amazon ECR
    - Create a Bedrock AgentCore runtime
    - Deploy your agent to the AgentCore Runtime

In [ ]:
### NOTE FOR NOW!!!!
### STOP STOP STOP ###
### You need to manually edit the Dockerfile to add Node.js installation ###
### STOP STOP STOP ###

# Edit the Docckerfile to add Node.js installation as below
# put it before the line that starts with " # Signal that this is running in Docker for host binding logic"

# Install Node.js (LTS) and npx for MCP server usage required by one of the agent tools
RUN apt-get update && apt-get install -y curl gnupg \
    && curl -fsSL https://deb.nodesource.com/setup_20.x | bash - \
    && apt-get install -y nodejs \
    && rm -rf /var/lib/apt/lists/*

In [ ]:
### Step 2: Deploy Your Agent to AgentCore Runtime
agentcore launch

#### After successful deployment, verify your setup by following these steps:
- Log in to the AWS Management Console
  - Navigate to the Amazon Bedrock AgentCore service
    - Confirm that your agent appears in the service dashboard
  - Navigate to the S3 service
    - See the new S3 bucket with your zip file of agent files
  - Navigate to the ECS service
    - In Repositories, see your ECR repo

In [ ]:
### Step 3: Check its status, note memory can take a while, and observability may take a while longer
agentcore status

# <span style="color:DarkSeaGreen">Invoke Your Agent</span>

- You can execute via CLI or using boto3 as provided below

In [ ]:
# random session id generator
import time
import random
import string

def generate_session_id(length=33):
    """
    Generate a session ID based on the current timestamp + random characters.
    
    Example output: "1738543290_ab4Xk2"
    """
    timestamp = int(time.time())  # current Unix timestamp
    rand_chars = ''.join(random.choices(string.ascii_letters + string.digits, k=length))
    return f"{timestamp}_{rand_chars}"


In [ ]:
### Via CLI
### Run this then copy the cell invokes (one at a time) to your terminal window to execute

# lets create a random session id
session_id = generate_session_id()

# lets give it some info about me to test short term memory later
user_query = "Hello, my name is Simon, you should always try and reference my name when you answer me"
print(f"agentcore invoke '{{\"prompt\": \"{user_query}\"}}' --session-id {session_id}")

# this should use the crypto_security_analyzer tool via its http_request strands tool
user_query = "How risky is the SHIBA token on Ethereum chain ID 1 with contract address 0x95aD61b0a150d79219dCF64E1E6Cc01f0B64C4cE"
print(f"agentcore invoke '{{\"prompt\": \"{user_query}\"}}' --session-id {session_id}")

# this should use the crypto_market_analyst tool via its mcp client strands tool
user_query = "Can you tell me the current price of Bitcoin?"
print(f"agentcore invoke '{{\"prompt\": \"{user_query}\"}}' --session-id {session_id}")

# this should use the crypto_educator tool via its retrieve strands tool
user_query = "What are the risks of investing in cryptocurrency?"
print(f"agentcore invoke '{{\"prompt\": \"{user_query}\"}}' --session-id {session_id}")

# this should use the general_knowledge tool via its default llm
user_query = "What is the capital of Wales and how many mountains does Wales have?"
print(f"agentcore invoke '{{\"prompt\": \"{user_query}\"}}' --session-id {session_id}")

user_query = "Thank you, good talking with you"
print(f"agentcore invoke '{{\"prompt\": \"{user_query}\"}}' --session-id {session_id}")

# stop the session to free resources and reduce costs
# NOTE this command seems to have been deprecated but leaving here for now
# Worst case, session will die after the idle-timeout configured above (eg 300 seconds = 5 minutes)
print(f"agentcore stop-session --session-id {session_id}")


### Now lets do the same but with boto3

In [ ]:
# via boto3
# get the arn of the agentcore runtime
acc = boto3.client('bedrock-agentcore-control', region_name=myRegion, verify=where())
ac = boto3.client('bedrock-agentcore', region_name=myRegion, verify=where())

# lets get the arn of our agent core runtime - this assume you did not give it a custom name when you configured it!!!
# so it looks for Strands_Orchestration in the name
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agentcore-control/client/list_agent_runtimes.html
runtimes = acc.list_agent_runtimes()
strands_arns = [
    rt['agentRuntimeArn'] 
    for rt in runtimes['agentRuntimes'] 
    if 'Strands_Orchestration' in rt['agentRuntimeArn']
]
agent_core_arn = strands_arns[0] if strands_arns else None

if not agent_core_arn:
    print("Error: Could not find AgentCore runtime ARN with 'Strands_Orchestration' in the name. Did you change it?")
else:
    print(f"Found AgentCore runtime ARN: {agent_core_arn}")
    print("✅ Done! Move to the next cell ->")

- you can change your query and run the cell below to invoke the agent as many times as you like
- it will all be done within your session maintaining short term memory for the agent

In [ ]:
session_id = generate_session_id()
print(f"Generated session ID: {session_id}")
print("You can use this to trace your session in CloudWatch logs and metrics.")
print("Under GenAI Observability, look for Bedrock AgentCore.")

print("✅ Done! Move to the next cell ->")

### You can rerun from here as much as you want changing the query below

In [ ]:
my_query = "What is the price of the top 5 coins"
print("✅ Done! Move to the next cell ->")

In [ ]:
if agent_core_arn:
    # invoke the agent via boto3
    # https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agentcore.html
    response = ac.invoke_agent_runtime(
        agentRuntimeArn=agent_core_arn,
        payload=json.dumps({"prompt": my_query}),
        runtimeSessionId=session_id
    )
    
    # Process and print the response
    if "text/event-stream" in response.get("contentType", ""):
        # Handle streaming response
        content = []
        for line in response["response"].iter_lines(chunk_size=10):
            if line:
                line = line.decode("utf-8")
                if line.startswith("data: "):
                    line = line[6:]
                    print(line)
                    content.append(line)
        print("\nComplete response:", "\n".join(content))

    elif response.get("contentType") == "application/json":
        # Handle standard JSON response
        content = []
        for chunk in response.get("response", []):
            content.append(chunk.decode('utf-8'))
        print(json.loads(''.join(content)))
    
    else:
        # Print raw response for other content types
        print(response)

print("✅ Done! Move to the next cell or change your query and run again ->")

# <span style="color:red">NEED TO FIX THE MCP TOOL<BR>ITS ERRORING with below<br>if its errored or timed out previously:</span>
Error: MCPClientInitializationError - the client session is currently running.

PRobably due to the way the mcp is created in the tool and he use of WITH


In [ ]:
# kill the session to free resources and reduce costs
ac.stop_runtime_session(
    agentRuntimeArn=agent_core_arn,
    runtimeSessionId=session_id
)

# <span style="color:DarkSeaGreen">Move to Lab 4</span>
# <span style="color:DarkSeaGreen">OR...</span>
# <span style="color:DarkSeaGreen">Clean Up Architecture</span>
### <span style="color:Red">Only do this if you have finished with this lab and any labs that depend on it!</span>
##### It will delete all architecture created, make sure you no longer need any of it!!!

In [ ]:
### run in terminal window under venv
# cleans up the resources created in the lab
# but not the s3 bucket
agentcore destroy

In [ ]:
# delete roles and policies
try:
    iam.detach_role_policy(
        RoleName=myPolicyAgentCore1, PolicyArn='arn:aws:iam::{}:policy/{}'.format(myAccountNumber, myRoleAgentCore)
    )
except Exception as e:
    print(f"Error detaching policy: {e}")   
    print(f"Deleted when you destroyed the AgentCore runtime using 'agentcore destroy' command")   

try:
    iam.delete_role(RoleName=myRoleAgentCore)
except Exception as e:
    print(f"Error deleting role: {e}")    
    print(f"Deleted when you destroyed the AgentCore runtime using 'agentcore destroy' command")   

try:
    iam.delete_policy(PolicyArn='arn:aws:iam::{}:policy/{}'.format(myAccountNumber, myPolicyAgentCore1))
except Exception as e:
    print(f"Error deleting policy: {e}")    
    print(f"Deleted when you destroyed the AgentCore runtime using 'agentcore destroy' command")   

print ('✅ Done! You have cleaned up the IAM role and policy created for AgentCore runtime')